In [8]:
import re
import matplotlib as plt
import pandas as pd
import numpy as np
import json
import matplotlib
import csv
from collections import defaultdict

# files
publication_name = "Siponen2020"
file_classified = open('./classification_first_iteration_curated/' + publication_name + '_classified', 'r')
file_unclassified = open('./classification_first_iteration_curated/' + publication_name + '_unclassified', 'r')

# patterns
pattern_sublime = '(?<=<t>)(.*?)(?=</t>)'
pattern_scraping = '(?<=<t>)(.*?)(?=<t>)'

# structures
distinctive_node_list = []
group_matches_dictionary = defaultdict(list)

# functions
def classified(line):
    if '<fc>' not in line:
        if '<t>' in line:
            return True

def get_title(line):
    title_match = None
    if ('</t>' in line):
        # tags created with sublime
        title_match = re.search(pattern_sublime, line)
    else:
        #tags created with the scraping algorithm
        title_match = re.search(pattern_scraping, line)
    if title_match == None:
        print("no closing tag: " + line)
    title = title_match.group(0)
    title = title.lower()
    return title

def construct_lists(file):
    count = 0
    current_parent_reference = ''
    for line in file:
        if (classified(line)):
            date = re.search("\d{4}", line)
            title = get_title(line)
            alphanumeric_filter = filter(str.isalnum, title)
            ref_id = "".join(alphanumeric_filter)
            if date != None:
                ref_id += date.group(0)
            if not (ref_id in distinctive_node_list):
                distinctive_node_list.append(ref_id)
            if '<PARENTARTICLE>' in line:
                count += 1
                current_parent_reference = ref_id
                continue
            group_matches_dictionary[current_parent_reference].append(ref_id)
    return count

if (construct_lists(file_classified) == construct_lists(file_unclassified)):
    print("created list successfully")
else:
    raise ValueError('number of parental nodes do not match')

#creating adjazenzlist in excel
with open('./adjazenz_lists/' + publication_name + '.csv', 'w') as output:
    writer = csv.writer(output)
    writer.writerow(["parentnode", "childnodes"])
    for parent, childlist in group_matches_dictionary.items():
        childlist_string = ''
        for child in childlist:
            childlist_string += child + ','
        writer.writerow([parent, childlist_string[0:-1]])


created list successfully
